In [486]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [487]:
import random
from imblearn.over_sampling import SMOTE

In [488]:
df = pd.read_csv("diabetes.csv")

In [489]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [490]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [491]:
# Split the data into features (X) and target variable (y)
X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # seed in random generation


In [492]:
# Train the logistic regression model
logistic_regression = LogisticRegression()

logistic_regression.fit(X_train, y_train)

c:\Users\yoges\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [493]:
# Predicting 
y_pred = logistic_regression.predict(X_test)
y_pred

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [494]:
print(y_test)

661    1
122    0
113    0
14     1
529    0
      ..
476    1
482    0
230    1
527    0
380    0
Name: Outcome, Length: 154, dtype: int64


In [495]:
f1_lr = f1_score(y_test, y_pred, average='binary')
print(f"f1 Score is {f1_lr}")

accuracy_mlp = accuracy_score(y_test, y_pred, normalize=True)
print(f"Accuracy is {accuracy_mlp}")

roc_mlp = roc_auc_score(y_test, y_pred, average='macro')
print(f"ROC Score is {roc_mlp}")

f1 Score is 0.6823529411764706
Accuracy is 0.8246753246753247
ROC Score is 0.7664545635315172


## 1. Random Undersampling:
    

##### Here we will select random instances (=count of minority class) from the majority class; hence undersampling it.

In [496]:
df['Outcome'].count()

np.int64(768)

In [497]:
class_counts = df['Outcome'].value_counts()
print(class_counts)

min_count = class_counts.min()
print("\nMinimum count is ",min_count)


Outcome
0    500
1    268
Name: count, dtype: int64

Minimum count is  268


In [498]:
# Randomly sample the dataset to balance it (undersampling) --> Won't affect minority class as sampling happening without replacement.
df_random_undersampling = df.groupby('Outcome').apply(lambda x: x.sample(min_count,replace=False)).reset_index(drop=True)

# Checking the size of each class after applying undersampling
print(df_random_undersampling['Outcome'].value_counts())


Outcome
0    268
1    268
Name: count, dtype: int64


C:\Users\yoges\AppData\Local\Temp\ipykernel_9700\1670489527.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_random_undersampling = df.groupby('Outcome').apply(lambda x: x.sample(min_count,replace=False)).reset_index(drop=True)


#### Checking accuracy post undersampling

In [499]:
# Split the data into features (X) and target variable (y)
X = df_random_undersampling.drop("Outcome", axis = 1)
y = df_random_undersampling["Outcome"]

# Splot the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # seed in random generation


In [500]:
# Train the logistic regression model
logistic_regression = LogisticRegression()

logistic_regression.fit(X_train, y_train)

c:\Users\yoges\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [501]:
# Metrics 
y_pred = logistic_regression.predict(X_test)

f1_lr = f1_score(y_test, y_pred, average='binary')
print(f"f1 Score is {f1_lr}")

accuracy_mlp = accuracy_score(y_test, y_pred, normalize=True)
print(f"Accuracy is {accuracy_mlp}")

roc_mlp = roc_auc_score(y_test, y_pred, average='macro')
print(f"ROC Score is {roc_mlp}")

f1 Score is 0.7777777777777778
Accuracy is 0.7777777777777778
ROC Score is 0.7788461538461539


##  2. Random oversampling

##### Here we will perform random oversampling on minority class.

In [502]:

# Count the number of observations in each class
class_counts = df['Outcome'].value_counts()
print(class_counts)

# Get the maximum class count (majority class count)
max_count = class_counts.max()
print(max_count)


Outcome
0    500
1    268
Name: count, dtype: int64
500


In [503]:
# Perform Random oversampling --> Here replace=True has been kept

oversampled_df = (df.groupby('Outcome')
                  .apply(lambda x: x.sample(max_count, replace=True))
                  .reset_index(drop=True))
# Note : Here, replace=True is set ==> The majority class will also be altered during sampling.

# Checking the size of each class after applying oversampling
class_sizes = oversampled_df['Outcome'].value_counts()
print(class_sizes)

Outcome
0    500
1    500
Name: count, dtype: int64


C:\Users\yoges\AppData\Local\Temp\ipykernel_9700\620856804.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(max_count, replace=True))


In [504]:
f1_lr = f1_score(y_test, y_pred, average='binary')
print(f"f1 Score is {f1_lr}")

accuracy_mlp = accuracy_score(y_test, y_pred, normalize=True)
print(f"Accuracy is {accuracy_mlp}")

roc_mlp = roc_auc_score(y_test, y_pred, average='macro')
print(f"ROC Score is {roc_mlp}")

f1 Score is 0.7777777777777778
Accuracy is 0.7777777777777778
ROC Score is 0.7788461538461539


## 3. Oversampling usign SMOT

In [505]:
# Split the data into features (X) and target variable (y)
X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

# Apply SMOTE to balance the dataset
smote = SMOTE()
X, y = smote.fit_resample(X, y)

# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # seed in random generation


In [506]:
# Train the logistic regression model
logistic_regression = LogisticRegression()

logistic_regression.fit(X_train, y_train)

c:\Users\yoges\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [507]:
# Viewing the Metrics
y_pred = logistic_regression.predict(X_test)

f1_lr = f1_score(y_test, y_pred, average='binary')
print(f1_lr)

accuracy_mlp = accuracy_score(y_test, y_pred, normalize=True)
print(accuracy_mlp)

roc_mlp = roc_auc_score(y_test, y_pred, average='macro')
print(roc_mlp)

0.8020833333333334
0.81
0.8100250626566415


## 4. Tomek Link (Undersampling)

In [508]:
from imblearn.under_sampling import TomekLinks

X = df.drop(columns=['Outcome'])  
y = df['Outcome']                 

tomek_links = TomekLinks()

X, y = tomek_links.fit_resample(X, y)

# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [509]:
# Train the logistic regression model
logistic_regression = LogisticRegression()

logistic_regression.fit(X_train, y_train)

c:\Users\yoges\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [510]:
# Viewing the Metrics
y_pred = logistic_regression.predict(X_test)

f1_lr = f1_score(y_test, y_pred, average='binary')
print(f1_lr)

accuracy_mlp = accuracy_score(y_test, y_pred, normalize=True)
print(accuracy_mlp)

roc_mlp = roc_auc_score(y_test, y_pred, average='macro')
print(roc_mlp)

0.7592592592592593
0.8181818181818182
0.7994802079168334


## 5. Class Weighting

In [511]:
# Split the data into features (X) and target variable (y)
X = df.drop("Outcome", axis = 1)
y = df["Outcome"]


# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # seed in random generation


In [512]:
from sklearn.linear_model import LogisticRegression
# Define class weights
class_weights = {0: 0.2, 1: 0.8}
# Create logistic regression object with class weights
lr = LogisticRegression(class_weight=class_weights)
# Train the model using the training data
lr.fit(X_train, y_train)

c:\Users\yoges\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight={0: 0.2, 1: 0.8})

In [513]:
# Predicting 
y_pred = logistic_regression.predict(X_test)

f1_lr = f1_score(y_test, y_pred, average='binary')
print(f1_lr)

accuracy_mlp = accuracy_score(y_test, y_pred, normalize=True)
print(accuracy_mlp)

roc_mlp = roc_auc_score(y_test, y_pred, average='macro')
print(roc_mlp)

0.7032967032967034
0.8246753246753247
0.7843507655597535


## In conclusion :

Best f1 score is of SMOTE Technique

Best accuracy is of Class Weight Balancing Technique

Best AUC score is of Tomek Link Technique